In [1]:
import boto3
import json
from functions.influxdb_manager import InfluxDBOperations

Python-dotenv could not parse statement starting at line 10
Python-dotenv could not parse statement starting at line 13
Python-dotenv could not parse statement starting at line 14
Python-dotenv could not parse statement starting at line 15
Python-dotenv could not parse statement starting at line 16
Python-dotenv could not parse statement starting at line 17
Python-dotenv could not parse statement starting at line 19
Python-dotenv could not parse statement starting at line 20
Python-dotenv could not parse statement starting at line 21


In [4]:
from ml_functions.model_serve import SageMakerHandler

sagemaker_handler = SageMakerHandler()
sagemaker_handler.get_data_and_predict()

Received response: {'predictions': [1]}
Complete. Return to the InfluxDB UI.


In [3]:
!source ~/.bashrc
import os
print(os.environ.get('AWS_ACCESS_KEY_ID'))

None


In [23]:
from dotenv import load_dotenv
import os
load_dotenv()

Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 2
Python-dotenv could not parse statement starting at line 3
Python-dotenv could not parse statement starting at line 5


False

In [4]:
global app_name
global region
aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
region = "eu-central-1"
model_role= "arn:aws:iam::401605265667:user/mluser"
app_name = 'stockmodel'

boto3.setup_default_session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region
)

In [14]:


influxDB = InfluxDBOperations()

input_df, predict_df = influxDB.get_data_from_influx()

In [6]:
def query_endpoint(app_name, input_json):
    client = boto3.session.Session().client('sagemaker-runtime', region)
    response = client.invoke_endpoint(
        EndpointName = app_name,
        Body = input_json,
        ContentType = 'application/json'
    )
    preds = response['Body'].read().decode('ascii')
    preds = json.loads(preds)
    print('Received response: {}'.format(preds))
    return preds

In [17]:
input_df.reset_index(drop=True, inplace=True)
query_input = input_df.to_dict(orient='split')
data = {"dataframe_split": query_input}
byte_data = json.dumps(data).encode('utf-8')
prediction = query_endpoint(app_name, input_json=byte_data)
predict_df["Target"] = prediction["predictions"][0]

influxDB.save_prediction_to_influx(predict_df)

Received response: {'predictions': [1]}
